In [18]:
import pandas as pd 
import xml.etree.ElementTree as et 
import os
import regex as re
import json
import os
import glob
from tqdm.autonotebook import tqdm

In [19]:
mmapfilelist = []
for root, dirs, files in os.walk("../../../Downloads/PostNLQAS"):
    mmapfilelist = mmapfilelist + glob.glob(root+'/*.mmap')


In [20]:
columns = ['filename', 'reffile', 'elempath', 'reffiletype', 'target', 'usedfunction', 'role', 'path', 'gid',\
           'fname', 'fns', 'type2' ,'type', 'path2','gid2', 'fname2', 'fns2']
print(len(columns))
df = pd.DataFrame(columns=columns)
df.head()

17


,filename,reffile,elempath,reffiletype,target,usedfunction,role,path,gid,fname,fns,type2,type,path2,gid2,fname2,fns2


In [21]:
len(mmapfilelist)

727

In [22]:
def parse_mmap_XML(xml_file): 
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    return iterxml(xroot, xml_file)
    
    
def iterxml(xml, filename):
    df = pd.DataFrame(columns=columns)
    for elem in list(xml): # 1
        path = ""
        firsttabs = ""
        rootelem = ""
        if "}" in elem.tag:
            rootelem = elem.tag.split("}", 1)[1]
        else:
            rootelem = elem.tag
            
        #print(firsttabs + rootelem)
        #print(firsttabs + "  " + str(elem.attrib))
        rootid = elem.attrib.get("id")
        rootname = elem.attrib.get("name")
        path += "/"+rootelem
        rootpath1 = path
        if rootelem == "generic" or rootelem == "content":
            for items in elem:# 2
                path = rootpath1
                firsttabs = "\t"
                tagname = ""
                if "}" in items.tag:
                    tagname = items.tag.split("}", 1)[1]
                else:
                    tagname = items.tag

                path += "/"+tagname
                #print(firsttabs + tagname)
                #print(firsttabs + "  " + str(items.attrib))
                #print(firsttabs + "  " + str(items.text))

                rootpath2 = path
                for inner in items: # 3
                    path = rootpath2
                    firsttabs = "\t\t"
                    tagname = ""
                    if "}" in inner.tag:
                        tagname = inner.tag.split("}", 1)[1]
                    else:
                        tagname = inner.tag

                    path += "/"+tagname
                    #print(firsttabs + tagname)
                    #print(firsttabs + "  " + str(inner.attrib))
                    #print(firsttabs + "  " + str(inner.text))
                    rootpath3 = path
                    for diep in inner: # 4
                        path = rootpath3
                        firsttabs = "\t\t\t"
                        tagname = ""
                        if "}" in diep.tag:
                            tagname = diep.tag.split("}", 1)[1]
                        else:
                            tagname = diep.tag

                        path += "/"+tagname
                        #print(firsttabs + tagname)
                        #print(firsttabs + "  " + str(diep.attrib))
                        #print(firsttabs + "  " + str(diep.text))
                        rootpath4 = path
                        for nogdieper in diep: # 5
                            path = rootpath4
                            firsttabs = "\t\t\t\t"
                            tagname = ""
                            if "}" in nogdieper.tag:
                                tagname = nogdieper.tag.split("}", 1)[1]
                            else:
                                tagname = nogdieper.tag

                            path += "/"+tagname
                            #print(firsttabs + tagname)
                            #print(firsttabs + "  " + str(nogdieper.attrib))
                            #print(firsttabs + "  " + str(nogdieper.text))
                            rootpath5 = path
                            passed = False
                            for evendeeper in nogdieper: # 6
                                path = rootpath5
                                firsttabs = "\t\t\t\t\t"
                                tagname = ""
                                if "}" in evendeeper.tag:
                                    tagname = evendeeper.tag.split("}", 1)[1]
                                else:
                                    tagname = evendeeper.tag

                                path += "/"+tagname
                                #print(firsttabs + tagname)
                                #print(firsttabs + "  " + str(evendeeper.attrib))
                                #print(firsttabs + "  " + str(evendeeper.text))
                                if not passed:
                                    reffile = evendeeper.text
                                    passed = True
                                if tagname == "elem":
                                    df = df.append({\
                                            'filename':filename, 'reffile':reffile,\
                                            'elempath':path,'reffiletype':nogdieper.attrib.get('typeID',None),\
                                            'usedfunction':evendeeper.text, \
                                            'role':inner.attrib.get('role',None),\
                                            'path':None,'gid':None,\
                                            'fname':None, 'fns':None,\
                                            'type2':None,'type':None,\
                                            'path2':None,'gid2':None,\
                                            'fname2':None, 'fns2':None},\
                                            ignore_index=True)
                                elif tagname == "transformation":
                                    rootpath6 = path
                                    for opnieuwevendeeper in evendeeper: # 7
                                        path = rootpath6
                                        firsttabs = "\t\t\t\t\t\t"
                                        tagname = ""
                                        if "}" in opnieuwevendeeper.tag:
                                            tagname = opnieuwevendeeper.tag.split("}", 1)[1]
                                        else:
                                            tagname = opnieuwevendeeper.tag

                                        path += "/"+tagname
                                        #print(firsttabs + tagname)
                                        #print(firsttabs + "  " + str(opnieuwevendeeper.attrib))
                                        #print(firsttabs + "  " + str(opnieuwevendeeper.text))
                                        if tagname == "brick":
                                            brickattr = opnieuwevendeeper.attrib
                                            rootpath7 = path
                                            for opnieuwevendeeper2 in opnieuwevendeeper: # 8
                                                path = rootpath7
                                                firsttabs = "\t\t\t\t\t\t\t"
                                                tagname = ""
                                                if "}" in opnieuwevendeeper2.tag:
                                                    tagname = opnieuwevendeeper2.tag.split("}", 1)[1]
                                                else:
                                                    tagname = opnieuwevendeeper2.tag

                                                path += "/"+tagname
                                                #print(firsttabs + tagname)
                                                #print(firsttabs + "  " + str(opnieuwevendeeper2.attrib))
                                                #print(firsttabs + "  " + str(opnieuwevendeeper2.text))
                                                rootpath8 = path
                                                for opnieuwevendeeper23 in opnieuwevendeeper2: # 9
                                                    path = rootpath8
                                                    firsttabs = "\t\t\t\t\t\t\t\t"
                                                    tagname = ""
                                                    if "}" in opnieuwevendeeper23.tag:
                                                        tagname = opnieuwevendeeper23.tag.split("}", 1)[1]
                                                    else:
                                                        tagname = opnieuwevendeeper23.tag

                                                    path += "/"+tagname
                                                    localattr = opnieuwevendeeper23.attrib
                                                    #print(firsttabs + tagname)
                                                    #print(firsttabs + "  " + str(opnieuwevendeeper23.attrib))
                                                    #print(firsttabs + "  " + str(opnieuwevendeeper23.text))
                                                    df = df.append({\
                                                        'filename':filename, 'reffile':None,\
                                                        'elempath':path,'reffiletype':None,\
                                                        'target':None, 'usedfunction':None, \
                                                        'role':None,\
                                                        'path':brickattr.get('path', None),'gid':brickattr.get('gid', None),\
                                                        'fname':brickattr.get('fname', None), 'fns':brickattr.get('fns', None),\
                                                        'type2':localattr.get('type',None),'type':brickattr.get('type',None),\
                                                        'path2':localattr.get('path', None),'gid2':localattr.get('gid', None),\
                                                        'fname2':localattr.get('fname', None), 'fns2':localattr.get('fns', None)},\
                                                        ignore_index=True)
    return df
        

In [23]:
file = '.mmap'
df = parse_mmap_XML(file)

FileNotFoundError: [Errno 2] No such file or directory: 'mm_BYD0500_BYD_GenericEnv_Level1and2_CI.mmap'

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 17 columns):
filename        0 non-null object
reffile         0 non-null object
elempath        0 non-null object
reffiletype     0 non-null object
target          0 non-null object
usedfunction    0 non-null object
role            0 non-null object
path            0 non-null object
gid             0 non-null object
fname           0 non-null object
fns             0 non-null object
type2           0 non-null object
type            0 non-null object
path2           0 non-null object
gid2            0 non-null object
fname2          0 non-null object
fns2            0 non-null object
dtypes: object(17)
memory usage: 0.0+ bytes


In [7]:
files = mmapfilelist
dfset = []
pbar = tqdm(total=len(mmapfilelist), leave=False)
pbar.set_description(desc='Processing mappings', refresh=True)

for i in files: 
    df = parse_mmap_XML(i)
    pbar.update(1)
#     print(len(df.columns))
#     df.dropna(axis=0, how='all', subset=df.columns.values, inplace=True)
    dfset.append(df)
pbar.close()

In [8]:
df = dfset[0]
pbar = tqdm(total=len(dfset), leave=False)
pbar.set_description(desc='Processing mappings', refresh=True)
pbar.update(1)
for i in range(1, len(dfset)):
    df = df.append(dfset[i])
    pbar.update(1)
pbar.close()
df.to_csv("../generateddata/QASmappings.csv")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 493071 entries, 0 to 7
Data columns (total 17 columns):
filename        493071 non-null object
reffile         6822 non-null object
elempath        493071 non-null object
reffiletype     6822 non-null object
target          0 non-null object
usedfunction    6719 non-null object
role            6822 non-null object
path            486249 non-null object
gid             486249 non-null object
fname           0 non-null object
fns             0 non-null object
type2           435930 non-null object
type            486249 non-null object
path2           419938 non-null object
gid2            419938 non-null object
fname2          15992 non-null object
fns2            15992 non-null object
dtypes: object(17)
memory usage: 67.7+ MB


In [1]:
# df.head()

In [11]:
filecombinations = {'file':[],'reffile':[]}
for filename in df['filename'].unique():
    splitdf = df[df['filename']==filename]
    for reffile in splitdf['reffile'].unique():
        if ".xsd" in str(reffile):
            filecombinations['file'].append(filename)
            filecombinations['reffile'].append(reffile)
#         filecombinations.append({filename:reffile})

In [12]:
filecombinationsdf = pd.DataFrame(filecombinations)

In [13]:
sub = lambda x: str(x).split("/")[-1]
interf = lambda x: str(x).split("/")[5]
filecombinationsdf['interface'] = filecombinationsdf['file'].apply(interf)
filecombinationsdf['mapping'] = filecombinationsdf['file'].apply(sub)

In [2]:
# filecombinationsdf.describe()

In [15]:
filecombinationsdf.to_csv("../generateddata/qasmapping_combinations.csv")

In [25]:
mappings = pd.read_csv("../generateddata/QASmappings.csv", sep=",")
mappings.drop(columns="Unnamed: 0", axis=1, inplace=True)

In [3]:
# mappings.head()

In [27]:
bricks = mappings[mappings['elempath'].str.contains("brick")]

In [4]:
# bricks.iloc[0]['filename']

In [29]:
bricks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486249 entries, 22 to 493062
Data columns (total 17 columns):
filename        486249 non-null object
reffile         0 non-null object
elempath        486249 non-null object
reffiletype     0 non-null object
target          0 non-null float64
usedfunction    0 non-null object
role            0 non-null object
path            486249 non-null object
gid             486249 non-null float64
fname           0 non-null float64
fns             0 non-null float64
type2           435930 non-null object
type            486249 non-null object
path2           419938 non-null object
gid2            419938 non-null float64
fname2          15992 non-null object
fns2            15992 non-null object
dtypes: float64(5), object(12)
memory usage: 66.8+ MB


In [173]:
def parse_mmap_bricks_XML(xml_file): 
    try:
        xtree = et.parse(xml_file)
        xroot = xtree.getroot()
    except et.ParseError:
        return None
    return iterxml(xroot, xml_file)
    
    
def iterxml(xml, filename):
    brickcount = 0
    brickbrickcount = 0
#     for elem in list(xml):
#         path = ""
#         firsttabs = ""
#         rootelem = ""
#         if "}" in elem.tag:
#             rootelem = elem.tag.split("}", 1)[1]
#         else:
#             rootelem = elem.tag
# #         if rootelem == "content":
#             #print(firsttabs + rootelem)
#             #print(firsttabs + "  " + str(elem.attrib))
#         rootid = elem.attrib.get("id")
#         rootname = elem.attrib.get("name")
#         path += "/"+rootelem
#         rootpath1 = path
#     try:
    content = xml.find('{urn:sap-com:xi}content')
    if content == None:
        return 0,0
    trafo = content.find("{urn:sap-com:xi:mapping:xitrafo}XiTrafo")
    if trafo == None:
        return 0,0
    meta = trafo.find("{urn:sap-com:xi:mapping:xitrafo}MetaData")
    if meta == None:
        return 0,0
    mapping = meta.find("{urn:sap-com:xi}mappingtool")
    if mapping == None:
        return 0,0
    project = mapping.find("{urn:sap-com:xi}project")
    if project == None:
        return 0,0
    transform = project.find("{urn:sap-com:xi}transformation")
    if transform == None:
        return 0,0
    for elem in transform:
        if elem.tag == "{urn:sap-com:xi}brick":
            brickcount += 1
            brickbrickcount += parse_mmap_brick(elem, filename)
#             else:
#                 iterxml(elem, filename, brickcount, brickbrickcount)
#     print(brickcount, brickbrickcount)
    return brickcount, brickbrickcount
#     except:
#         return 0,0
            
def parse_mmap_brick(brick, filename):
    return len(brick.findall('.//{urn:sap-com:xi}brick'))
#     print("hurr")
#     for elem in list(brick): # 1
#         rootelem = ""
#         if "}" in elem.tag:
#             rootelem = elem.tag.split("}", 1)[1]
#         else:
#             rootelem = elem.tag
            
#         print(firsttabs + rootelem)
#         print(firsttabs + "  " + str(elem.attrib))
#         if rootelem == "brick":
#             print("brick")
#             brickcount += 1
# #         firsttabs +="\t"
# #         path += rootelem + "/"
# #         if len(list(elem)) >= 1:
#             brickcount += parse_mmap_brick(elem, filename, firsttabs, path, brickcount)
#         if len(list(elem)) == 0:
#             return brickcount

In [169]:
parse_mmap_bricks_XML("../../../Downloads/PostNLQAS//src/main/resources/mapping/.mmap")

289 252


(289, 252)

In [170]:
brickfilelist = bricks['filename'].unique()

In [175]:
collection = []
for i in mmapfilelist:
    val = parse_mmap_bricks_XML(i)
    if val != None:
        collection.append({'file':i, "bricks":val[0], "sources":val[1]})

In [176]:
brickdf = pd.DataFrame(collection)

In [5]:
# brickdf.head()

In [6]:
mmap_complexity_df = pd.read_csv("../generateddata/QAScomplexity-mappings-xml.csv")
mmap_complexity_df.drop(columns=['Unnamed: 0', "complexity", 'concept_complexity'], inplace=True)
mmap_complexity_df.head()

In [7]:
combineddf = pd.merge(brickdf, mmap_complexity_df, on="file")
combineddf.head()

In [198]:
combineddf.to_csv("../generateddata/QASmmap-stats.csv")

In [199]:
combineddf.describe()

,bricks,sources,size_complexity,name_complexity
count,726.000000,726.000000,726.000000,726.000000
mean,663.571625,673.435262,0.189785,0.262346
std,3387.305816,3359.790633,0.267399,0.337922
min,0.000000,0.000000,0.016954,0.000000
25%,0.000000,0.000000,0.018198,0.000000
50%,12.000000,19.000000,0.068031,0.092045
75%,48.000000,110.750000,0.215382,0.347009
max,30757.000000,30785.000000,1.000000,1.000000
